In [57]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.utils.class_weight import compute_class_weight
from sklearn import preprocessing
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import accuracy_score, roc_auc_score,matthews_corrcoef,f1_score, brier_score_loss,average_precision_score, confusion_matrix,mean_squared_error,r2_score,precision_score
from sklearn.linear_model import LogisticRegression,ElasticNet
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import pickle
import scipy.stats as sts
import json
from sklearn.utils import check_X_y, safe_mask
import functools
from joblib import Parallel, delayed
from multiprocessing import cpu_count
from torch.utils.data import Dataset
import torch
import torch_geometric
from sklearn.decomposition import PCA
import networkx as nx
from sklearn.calibration import calibration_curve
import warnings
import sklearn.exceptions as ske
warnings.simplefilter("ignore", category=ske.ConvergenceWarning)
from pykeen.datasets import PrimeKG
import torch

# Load dataset

In [49]:
# load dataset
dataset_clinical_feature = pd.read_excel("./dataSet.xlsx",dtype='object')

continue_features = ['Age','Height', 'Weight', 'BMI', 'Systolic_blood_pressure', 'Diastolic_blood_pressure',
                               'Heart_rate', 'Abdominal_circumference', 'Pelvis_circumference', 'Oxygen_saturation',
                               'Years_as_a_smoker', 'Leukocytes', 'Erythrocytes', 'Haemoglobin', 'Haematocrit', 'MCV',
                               'MCH', 'MCHC', 'RDW', 'Platelets', 'MPV',
                                'Basophils_#', 'Neutrophils_#', 'Lymphocytes_#', 'Monocytes_#',
                               'Eosinophils_#', 'Blood_sugar', 'Uric_acid', 'GGT', 'Total_bilirubin',
                               'hsTnI', 'Triglycerides', 'Total_cholesterol', 'HDL_cholesterol', 'LDL_cholesterol',
                               'CRP', 'hs_CRP', 'HbA1c_IFCC']

ordinal_features = [ 'Score_FSSQ', 'Score_PHQ_9_4', 'Score_PREDIMED']

binary_features = ['Sex','Family_history_of_cerebrovascular_disease',
                                  'Family_history_of_cardiovascular_disease', 'Hypertension', 'Diabetes_mellitus',
                                  'Hypercholesterolemia','Peripheral_artery_disease', 'Anaemia','Chronic kidney disease',
                                  'Thrombophilia','Statins', 'NOA', 'Beta_blockers', 'ACE_Inhibitors', 'Sartans',
                                  'Diuretics', 'Nitrates', 'Calcium_antagonists_dihydropyridines',
                                  'Calcium_antagonists_not_dihydropyridines', 'ASA', 'Clopidogrel', 'Warfarin',
                                  'Oral_hypoglycemics', 'Insulin', 'Anxiolytic_Antidepressant', 'n_3_PUFA', 'Ezetimibe',
                                  'Menopause_andropause', 'Effects_of_other_external_causes',
                                  'Gastritis_and_duodenitis', 'Intervertebral_disc_disorders',
                                  'Hereditary_hemolytic_anemias', 'Hypothyroidism',
                                  'Osteoporosis_osteopenia_and_pathological_fracture', 'Cholelithiasis_and_cholecystitis',
                                  'Allergic_rhinitis', 'Thyroiditis', 'Depression', 'Glaucoma', 'Diseases_of_esophagus',
                                  'Nontoxic_nodular_goiter', 'Thyrotoxicosis_with_or_without_goiter', 'Asthma',
                                  'Chronic_airway_obstruction', 'Viral_hepatitis',
                                  'Upper_gastrointestinal_congenital_anomalies', 'Parkinsons_disease',
                                  'Chronic_liver_disease_and_cirrhosis', 'Diverticulosis_and_diverticulitis',
                                  'Psoriasis_and_related_disorders', 'Hyperplasia_of_prostate','Sleep_disorders',
                                  'Disorders_of_protein_plasma_amino_acid_transport_and_metabolism','Migraine',
                                  'Other_diseases_of_blood_and_blood_forming_organs',
                                  'Nonspecific_findings_on_examination_of_blood', 'Other_disorders_of_metabolism']

categorical_features = ['Level_of_education', 'Marital_status', 'Work', 'Physical_activity_frequency',
                                      'Smoking_habits', 'Smoking_daily_quantity']

labels = ['Stenosis_area_percentage']

# Utils function for training:

### 1) discretize the Coronary Artery Stenosis to create the label classification


In [50]:
def create_label_dataset(dataset,label):
    
    # add a cad group
    cad_group = []
    for index,row in dataset.iterrows():
        if row['Stenosis_area_percentage'] == 0:
            cad_group.append(0)
        elif (row['Stenosis_area_percentage'] > 0) and (row['Stenosis_area_percentage'] < 25):
            cad_group.append(1)
        elif (row['Stenosis_area_percentage'] >= 25) and (row['Stenosis_area_percentage'] < 50):
            cad_group.append(2)
        elif (row['Stenosis_area_percentage'] >= 50) and (row['Stenosis_area_percentage'] < 70):
            cad_group.append(3)
        elif (row['Stenosis_area_percentage'] >= 70):
            cad_group.append(4)

    dataset['cad_group'] = cad_group

    # discretize the label
    if label == "2vsall":
        dataset['cad_group'] = dataset['cad_group'].replace(0,0)
        dataset['cad_group'] = dataset['cad_group'].replace(1,0)
        dataset['cad_group'] = dataset['cad_group'].replace(2,1)
        dataset['cad_group'] = dataset['cad_group'].replace(3,0)
        dataset['cad_group'] = dataset['cad_group'].replace(4,0)
    elif label == "m2vsall":
        dataset['cad_group'] = dataset['cad_group'].replace(0,0)
        dataset['cad_group'] = dataset['cad_group'].replace(1,0)
        dataset['cad_group'] = dataset['cad_group'].replace(2,1)
        dataset['cad_group'] = dataset['cad_group'].replace(3,1)
        dataset['cad_group'] = dataset['cad_group'].replace(4,1)
    elif label == "3vsall":
        dataset['cad_group'] = dataset['cad_group'].replace(0,0)
        dataset['cad_group'] = dataset['cad_group'].replace(1,0)
        dataset['cad_group'] = dataset['cad_group'].replace(2,0)
        dataset['cad_group'] = dataset['cad_group'].replace(3,1)
        dataset['cad_group'] = dataset['cad_group'].replace(4,0)

    del dataset['Stenosis_area_percentage']
            
    return dataset

### 2) variables encoding for making categorical as one-hot and standardized numerical


In [ ]:
def scale_numerical_encode_categorical(X_train, X_val, binary_features, categorical_features,continue_features,label):
    
    if ((len(binary_features) == 0) and (len(categorical_features) == 0)):
        
        # numerical and ordinal scaled with standardscaler
        standard_scaler = preprocessing.StandardScaler()
        # TRAIN
        df_train_numerical = X_train.loc[:, X_train.columns.isin(continue_features)].copy(True)
        X_train_numerical_scaled = standard_scaler.fit_transform(df_train_numerical)
        df_train_numerical_scaled = pd.DataFrame(X_train_numerical_scaled,columns=df_train_numerical.columns)
        df_train_numerical_scaled['Sample_ID'] = X_train['Sample_ID']
        df_train_numerical_scaled[label] = X_train[label]
        # VAL
        df_val_numerical = X_val.loc[:, X_val.columns.isin(continue_features)].copy(True)
        X_val_numerical_scaled = standard_scaler.transform(df_val_numerical)
        df_val_numerical_scaled = pd.DataFrame(X_val_numerical_scaled,columns=df_val_numerical.columns)
        df_val_numerical_scaled['Sample_ID'] = X_val['Sample_ID']
        df_val_numerical_scaled[label] = X_val[label]
        
        
        X_train = df_train_numerical_scaled.loc[:, ~df_train_numerical_scaled.columns.isin(['Sample_ID','cad_group'])]
        y_train = df_train_numerical_scaled['cad_group']
        X_train.columns = X_train.columns.astype(str)

        X_val = df_val_numerical_scaled.loc[:, ~df_val_numerical_scaled.columns.isin(['Sample_ID','cad_group'])]
        y_val = df_val_numerical_scaled['cad_group']
        X_val.columns = X_val.columns.astype(str)
        
        return df_train_numerical_scaled, df_val_numerical_scaled, X_train, y_train, X_val, y_val

    else:
        
        # numerical and ordinal scaled with standardscaler
        standard_scaler = preprocessing.StandardScaler()
        # TRAIN
        df_train_numerical = X_train.loc[:, X_train.columns.isin(continue_features)].copy(True)
        X_train_numerical_scaled = standard_scaler.fit_transform(df_train_numerical)
        df_train_numerical_scaled = pd.DataFrame(X_train_numerical_scaled,columns=df_train_numerical.columns)
        df_train_numerical_scaled['Sample_ID'] = X_train['Sample_ID']
        df_train_numerical_scaled[label] = X_train[label]
        # VAL
        df_val_numerical = X_val.loc[:, X_val.columns.isin(continue_features)].copy(True)
        X_val_numerical_scaled = standard_scaler.transform(df_val_numerical)
        df_val_numerical_scaled = pd.DataFrame(X_val_numerical_scaled,columns=df_val_numerical.columns)
        df_val_numerical_scaled['Sample_ID'] = X_val['Sample_ID']
        df_val_numerical_scaled[label] = X_val[label]
    
        #### binary values as K-1 values
        # TRAIN
        df_train_binary_dummies = pd.get_dummies(X_train[binary_features].astype(str),drop_first=True)
        # VAL
        df_val_binary_dummies = pd.get_dummies(X_val[binary_features].astype(str),drop_first=True)

        union_binary_features = list(set(df_train_binary_dummies.columns).union(set(df_val_binary_dummies.columns)))
        df_train_binary_dummies = df_train_binary_dummies.reindex(columns=union_binary_features, fill_value=0)
        df_val_binary_dummies = df_val_binary_dummies.reindex(columns=union_binary_features, fill_value=0)

        df_train_binary_dummies['Sample_ID'] = X_train['Sample_ID']
        df_train_binary_dummies[label] = X_train[label]
        df_val_binary_dummies['Sample_ID'] = X_val['Sample_ID']
        df_val_binary_dummies[label] = X_val[label]

        #### categorical as K values
        # TRAIN
        df_train_categorical_dummies = pd.get_dummies(X_train[categorical_features].astype(str),drop_first=False)

        # VAL
        df_val_categorical_dummies = pd.get_dummies(X_val[categorical_features].astype(str),drop_first=False)

        union_categorical_features = list(set(df_train_categorical_dummies.columns).union(set(df_val_categorical_dummies.columns)))
        df_train_categorical_dummies = df_train_categorical_dummies.reindex(columns=union_categorical_features, fill_value=0)
        df_val_categorical_dummies = df_val_categorical_dummies.reindex(columns=union_categorical_features, fill_value=0)

        df_train_categorical_dummies['Sample_ID'] = X_train['Sample_ID']
        df_train_categorical_dummies[label] = X_train[label]
        df_val_categorical_dummies['Sample_ID'] = X_val['Sample_ID']
        df_val_categorical_dummies[label] = X_val[label]
    
        new_training = (df_train_binary_dummies.merge(df_train_categorical_dummies,on=["Sample_ID",label])).merge(df_train_numerical_scaled,on=["Sample_ID",label])
        new_val = (df_val_binary_dummies.merge(df_val_categorical_dummies,on=["Sample_ID",label])).merge(df_val_numerical_scaled,on=["Sample_ID",label])
            
        X_train = new_training.loc[:, ~new_training.columns.isin(['Sample_ID','cad_group'])]
        y_train = new_training['cad_group']
        X_train.columns = X_train.columns.astype(str)

        X_val = new_val.loc[:, ~new_val.columns.isin(['Sample_ID','cad_group'])]
        y_val = new_val['cad_group']
        X_val.columns = X_val.columns.astype(str)
        
        
        
        return new_training, new_val, X_train, y_train, X_val, y_val

### 3) compute metric for model evaluation

In [ ]:
def eval_model_compute_metrics(model, X, y_true, dataframe_results, n_th_iteration,i_th_fold,j_th_fold,
                               parameters_combination, type_validation, task):


    # make prediction
    y_pred = model.predict(X)
    
    y_pred_prob = model.predict_proba(X)
    
    mcc = matthews_corrcoef(y_true, y_pred)
    wf1 = f1_score(y_true, y_pred,average='weighted')
    auc = roc_auc_score(y_true, y_pred_prob[:,1])
    prec = precision_score(y_true, y_pred,zero_division=0.0)
    
    if type_validation == "val":

        # append a row on the dataframe
        new_row = pd.Series({"n_repetition":n_th_iteration,
                                "outer_fold":i_th_fold,
                                "inner_fold":j_th_fold,
                                "hyperparameters_combination":str(parameters_combination),
                                "validation_mcc":mcc,
                                "validation_wROCAUC":auc,
                                    "validation_wf1":wf1,
                                    "validation_prec_ppv":prec})

        dataframe_results =   pd.concat([dataframe_results, new_row.to_frame().T], ignore_index=True)

    elif type_validation == "test":
        accuracy = accuracy_score(y_true, y_pred)
        aupr = average_precision_score(y_true, y_pred_prob[:,1])
        auc = roc_auc_score(y_true, y_pred_prob[:,1])
        brier = brier_score_loss(y_true, y_pred_prob[:,1])

        cf = confusion_matrix(y_true, y_pred)
        cf_string = "TP: " + str(cf[1][1]) + " FP: " + str(cf[0][1]) + "TN: " + str(cf[0][0]) + " FN: " + str(cf[1][0])
        metrics_sens = str(cf[1][1] / (cf[1][1]+cf[1][0]))
        metrics_spec = str(cf[0][0] / (cf[0][0]+cf[0][1]))
        metrics_prec = str(cf[1][1] / (cf[1][1]+cf[0][1]))
        metrics_npv = str(cf[0][0] / (cf[0][0]+cf[1][0]))    

        # append a row on the dataframe
        new_row = pd.Series({"n_repetition":n_th_iteration,
                                "outer_fold":i_th_fold,
                                "hyperparameters_combination":str(parameters_combination),
                                "test_accuracy":accuracy,
                                "test_wROCAUC":auc,
                                "test_mcc":mcc,
                                "test_wPRAUC":aupr,
                                "test_wF1":wf1,
                                "test_brier": brier,
                                "test_confusion_matrix":cf_string,
                                "test_sens_rec":metrics_sens,
                                "test_spec":metrics_spec,
                                "test_prec_ppv":metrics_prec,
                                "test_npv":metrics_npv})

        dataframe_results =   pd.concat([dataframe_results, new_row.to_frame().T], ignore_index=True)
            
    return dataframe_results

### 4) transpose the gene dataset

In [ ]:
def transpose_and_index_gene_df(gene_df):
    gene_expr_data = gene_df.copy(True)
    gene_expr_data = gene_expr_data.T
    gene_expr_data['Sample_ID'] = gene_expr_data.index
    gene_expr_data.reset_index(drop=True)
    return gene_expr_data

### 5) create the patients average representation

In [ ]:
### CLINICAL FEATURES

def associate_patient_annotations_to_clinical_embeddings(node_df,
                                                patient_id, 
                                                clinical_features_annotation,
                                                entities_embeddings):
    
    patient_tensor = torch.zeros(len(clinical_features_annotation[patient_id]),2*entities_embeddings.shape[1])
    
    count_annotation = 0
    
    # clinical features
    for clinical_annotation in clinical_features_annotation[patient_id]:
        
        # single annotation
        if len(clinical_annotation)==1:
            
            if clinical_annotation[0] == "Family history of cerebrovascular disease":
                if "Family history of heart disease" in clinical_annotation:
                    continue
                else:
                    clinical_annotation_to_search = "Family history of heart disease"
            else:
                clinical_annotation_to_search = clinical_annotation[0]
            
            # extract nodeID
            nodeID_annotation = node_df[node_df['node_name'] == clinical_annotation_to_search]['Node ID'].iloc[0]
            
            # extract the embedding from the pretrained model
            patient_tensor[count_annotation] = torch.concatenate([torch.real(entities_embeddings[nodeID_annotation]),torch.imag(entities_embeddings[nodeID_annotation])])
            
        # multiple annotation
        else:
            
            intermediate_annotation_tensor = torch.zeros(len(clinical_annotation),2*entities_embeddings.shape[1])
            
            for i_th, clinical_annotation_th in enumerate(clinical_annotation):
                nodeID_annotation = node_df[node_df['node_name'] == clinical_annotation_th]['Node ID'].iloc[0]
                
                # extract the embedding from the pretrained model
                intermediate_annotation_tensor[i_th] = torch.concatenate([torch.real(entities_embeddings[nodeID_annotation]),torch.imag(entities_embeddings[nodeID_annotation])])
                
            # simply average the intermediate annotation tensor
            patient_tensor[count_annotation] = torch.mean(intermediate_annotation_tensor,axis=0)
            
        count_annotation+=1
            
    return patient_tensor


#for the genes, the returned tensor depends on the variables used
def from_gene_list_to_matrix_embedding(gene_list, entities_embeddings, dict_gene_alias):
    
    gene_var_tensor = torch.zeros(len(gene_list),2*entities_embeddings.shape[1])
    
    for count_gene, gene in enumerate(gene_list):
        
        gene_to_search = dict_gene_alias_mapping_primeKG[gene]
          
        if len(node_df[node_df['node_name'] == gene_to_search]['Node ID']) == 0:
            gene_var_tensor[count_gene] = torch.ones(1,2*entities_embeddings.shape[1])
        else:
            nodeID_annotation = node_df[node_df['node_name'] == gene_to_search]['Node ID'].iloc[0]
            gene_var_tensor[count_gene] = torch.concatenate([torch.real(entities_embeddings[nodeID_annotation]),torch.imag(entities_embeddings[nodeID_annotation])])

    return gene_var_tensor


def make_patient_representation(dataset, annotation_embedding,  gene_expr_features,
                             dict_gene_alias_mapping_primeKG,
                             dict_Sample_ID_clinical_annotation_tensor,
                             clinical_features_not_mapped):
    
    dataset_tensor = torch.zeros(dataset.shape[0],2*annotation_embedding.shape[1] + len(clinical_features_not_mapped))
    gene_var_tensor = from_gene_list_to_matrix_embedding(gene_expr_features, annotation_embedding,dict_gene_alias_mapping_primeKG)
        
    for count_id, patient_id in enumerate(dataset['Sample_ID']):
        patient_clinical_var_tensor = dict_Sample_ID_clinical_annotation_tensor[patient_id]
                    
        patient_gene_expression_values = torch.tensor(dataset[dataset['Sample_ID'] == patient_id][gene_expr_features].values).reshape(-1,1)
        patient_gene_expression_tensor = torch.mul(patient_gene_expression_values,gene_var_tensor)
            
        clinical_values_not_mapped = torch.tensor(dataset[dataset['Sample_ID'] == patient_id][clinical_features_not_mapped].values).reshape(-1)
            
        patient_tensor = torch.mean(torch.concatenate([patient_clinical_var_tensor,patient_gene_expression_tensor],axis=0),axis=0)
    
        patient_tensor = torch.concatenate([clinical_values_not_mapped,patient_tensor],axis=0)
            
        dataset_tensor[count_id] = patient_tensor
    
    return dataset_tensor



### 6) load and extract pykeen embeddings

In [ ]:
def read_model_return_embeddings(model_path, dataset):

    trained_best_model = torch.load(model_path)

    entity_representation_modules: List['pykeen.nn.Representation'] = trained_best_model.entity_representations
    relation_representation_modules: List['pykeen.nn.Representation'] = trained_best_model.relation_representations

    entity_embeddings: pykeen.nn.Embedding = entity_representation_modules[0]
    relation_embeddings: pykeen.nn.Embedding = relation_representation_modules[0]

    entity_embedding_tensor: torch.FloatTensor = entity_embeddings().detach().cpu()
    relation_embedding_tensor: torch.FloatTensor = relation_embeddings().detach().cpu()

    entity_to_id = dataset.entity_to_id
    relation_to_id = dataset.relation_to_id

    return trained_best_model, entity_embedding_tensor, relation_embedding_tensor, entity_to_id, relation_to_id


# reading file for KG embeddings strategy

In [51]:
# read the PrimeKG node file
nodes = pd.read_csv("./PrimeKG/nodes.csv")

# define the path with the results
results_path = "./"

# RotatE embeddings
best_RotatE_path = results_path + '/RotatE/200/best_model/replicates/replicate-00000/trained_model.pkl'
best_RotatE_model, best_RotatE_embeddings_entities, best_RotatE_embeddings_relations, entity_to_id, relation_to_id = read_model_return_embeddings(best_RotatE_path, PrimeKG())

# prepare the node df embeddings
node_df = pd.DataFrame.from_dict({'node_name': list(entity_to_id.keys()),'Node ID': list(entity_to_id.values())})
node_df = node_df.merge(nodes[['node_name','node_type']],on='node_name',how="left")
node_more_type = list(node_df['node_name'].value_counts()[node_df['node_name'].value_counts().values>1].index)

# read the annotation from INTESTRATCAD
with open('./clinical_annotations.pkl', 'rb') as fp:
        dict_patient_clinical_feature_annotation_NOTincluding_normal_PrimeKG = pickle.load(fp) 
        
import pickle
with open('./dict_gene_alias_mapping_primeKG.pkl', 'rb') as fp:
        dict_gene_alias_mapping_primeKG = pickle.load(fp) 

## CLINICAL FEATURES
dict_Sample_ID_clinical_annotation_tensor = {}

for id_patient in tqdm(dataset['Sample_ID']):
    dict_Sample_ID_clinical_annotation_tensor[id_patient] = associate_patient_annotations_to_clinical_embeddings(node_df, 
                                                                 id_patient, 
                                           dict_patient_clinical_feature_annotation_NOTincluding_normal_PrimeKG,
                                           best_RotatE_embeddings_entities    )


# define the clinical features not mapped to concatenate as values to the final tensor
# note that the reported variables name is the one obtained after the one-hot encoding
clinical_features_not_mapped = ['Sex_1','Age','Weight','Height','Smoking_daily_quantity_0','Smoking_daily_quantity_1',
                                'Smoking_daily_quantity_2','Smoking_daily_quantity_3','Physical_activity_frequency_0',
                                'Physical_activity_frequency_1','Physical_activity_frequency_2',
                                'Physical_activity_frequency_3','Work_0','Work_1','Work_2','Marital_status_0',
                                'Marital_status_1','Marital_status_2','Level_of_education_0','Level_of_education_1',
                                'Level_of_education_2']
        


# function for classification with ML

In [71]:
def ML_classification(random_state, classifier, path_splits, n_repetition, path_output, label, nfold,
                     source, KG_emb, annotation_embedding):
    
    print(classifier)
    dataframe_results_val = pd.DataFrame(columns = ["n_repetition","outer_fold","inner_fold",
                                                    "hyperparameters_combination","validation_mcc",
                                                    "validation_wROCAUC","validation_wf1","validation_prec_ppv"])
    dataframe_results_test = pd.DataFrame(columns = ["n_repetition","outer_fold","hyperparameters_combination","test_accuracy","test_wROCAUC",
                                                     "test_mcc","test_wPRAUC","test_wF1","test_brier",
                                    "test_confusion_matrix", "test_sens_rec", "test_spec","test_prec_ppv", "test_npv"])
    
    # repetition loop
    for n in range(n_repetition):
        rn = random_state + n
        
        # outer cross validation loop 
        for i in tqdm(range(nfold)):

            # inner cross validation loop 
            for j in range(nfold):
                
                
                if source == "clinical_variables":
                    training_set = create_label_dataset(pd.read_excel(path_splits + "dataset_training_" + str(rn) + "_" + str(i) + "_" + str(j) + ".xlsx"),label)
                    validation_set = create_label_dataset(pd.read_excel(path_splits + "dataset_val_" + str(rn) + "_" + str(i) + "_" + str(j) + ".xlsx"),label)


                    # scale and dummy encodings, define training dataset and outcome
                    training_set_scaled, validation_set_scaled, X_train, y_train, X_val, y_val = scale_numerical_encode_categorical(training_set, validation_set, 
                                                                        binary_features, categorical_features,
                                                                        continue_features + ordinal_features,
                                                                        "cad_group")
                    
                elif source == "gene_expression":

                    training_set = create_label_dataset(pd.read_excel(path_splits + "dataset_training_" + str(rn) + "_" + str(i) + "_" + str(j) + ".xlsx"),label)
                    validation_set = create_label_dataset(pd.read_excel(path_splits + "dataset_val_" + str(rn) + "_" + str(i) + "_" + str(j) + ".xlsx"),label)

                    training_set_gen = transpose_and_index_gene_df(pd.read_csv(path_splits + "dataset_gen_train_" + str(rn) + "_" + str(i) + "_" + str(j) + '.txt' , sep="\t"))
                    validation_set_gen = transpose_and_index_gene_df(pd.read_csv(path_splits + "dataset_gen_val_" + str(rn) + "_" + str(i) + "_" + str(j) + '.txt' , sep="\t"))

                    training_set = training_set_gen.merge(training_set[['Sample_ID','cad_group']].sort_values(by="Sample_ID"), on ="Sample_ID")
                    validation_set = validation_set_gen.merge(validation_set[['Sample_ID','cad_group']].sort_values(by="Sample_ID"), on ="Sample_ID")

                    gene_expr_features = list(set(training_set.columns).difference(set(['Sample_ID','cad_group'])))

                    # scale and dummy encodings, define training dataset and outcome
                    training_set_scaled, validation_set_scaled, X_train, y_train, X_val, y_val = scale_numerical_encode_categorical(training_set, validation_set,  
                                                                            [], [], gene_expr_features, "cad_group")


                elif source == "clinical_gene_expression":

                    # clinical var
                    training_set_cli = create_label_dataset(pd.read_excel(path_splits + "dataset_training_" + str(rn) + "_" + str(i) + "_" + str(j) + ".xlsx"),label)
                    validation_set_cli = create_label_dataset(pd.read_excel(path_splits + "dataset_val_" + str(rn) + "_" + str(i) + "_" + str(j) + ".xlsx"),label)

                    training_set_gen = transpose_and_index_gene_df(pd.read_csv(path_splits + "dataset_gen_train_" + str(rn) + "_" + str(i) + "_" + str(j) + '.txt' , sep="\t"))
                    validation_set_gen = transpose_and_index_gene_df(pd.read_csv(path_splits + "dataset_gen_val_" + str(rn) + "_" + str(i) + "_" + str(j) + '.txt' , sep="\t"))

                    training_set = training_set_gen.merge(training_set_cli.sort_values(by="Sample_ID"), on ="Sample_ID")
                    validation_set = validation_set_gen.merge(validation_set_cli.sort_values(by="Sample_ID"), on ="Sample_ID")


                    with open(path_splits + 'gene_selected_DEG_' + str(rn) + "_" + str(i) + "_" + str(j) + '.json', 'r') as file:
                        gene_expr_features = json.load(file)

                    training_set_scaled, validation_set_scaled, X_train, y_train, X_val, y_val = scale_numerical_encode_categorical(training_set, validation_set, 
                                                                                    binary_features, categorical_features,
                                                                                    continue_features + ordinal_features+gene_expr_features,
                                                                                    "cad_group")
                    
                    
                    if KG_emb:

                        X_train = torch.Tensor.numpy(make_patient_representation(training_set_scaled, 
                                                                            annotation_embedding,  
                                                                            gene_expr_features,
                                                                            dict_gene_alias_mapping_primeKG,
                                                                            dict_Sample_ID_clinical_annotation_tensor,
                                                                            clinical_features_not_mapped))

                        X_val = torch.Tensor.numpy(make_patient_representation(validation_set_scaled, 
                                                                            annotation_embedding,  
                                                                            gene_expr_features,
                                                                            dict_gene_alias_mapping_primeKG,
                                                                            dict_Sample_ID_clinical_annotation_tensor,
                                                                            clinical_features_not_mapped))


                # compute class weight
                class_weight = compute_class_weight('balanced',classes=np.unique(y_train), y = y_train)
                dict_class_weight = {}
                for c in np.unique(y_train):
                    dict_class_weight[c] = class_weight[c]

                # create classifier for each combination in a grid search
                if classifier == "logistic_regression":
                    parameters_grid = {'C': [0.0001, 0.001, 0.01, 0.1, 1, 10 , 20, 50, 100]}
                    grid_search = ParameterGrid(parameters_grid)

                    for parameters_combination in grid_search:

                        model = LogisticRegression(penalty="l1", solver = "liblinear", random_state = rn,
                                                    C = parameters_combination['C'],
                                                    class_weight = dict_class_weight,
                                                    max_iter = 1000)

                        model.fit(X_train, y_train)
                        dataframe_results_val = eval_model_compute_metrics(model, X_val, y_val, dataframe_results_val, 
                                                                               n,i,j,parameters_combination, "val","classification")

                elif classifier == "elastic_net":
                    parameters_grid = {'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 20, 50, 100],
                                    'l1_ratio': [0.25, 0.5, 0.75]}
                    grid_search = ParameterGrid(parameters_grid)

                    for parameters_combination in grid_search:

                        model = LogisticRegression(penalty="elasticnet", solver="saga", random_state = rn,
                                                l1_ratio = parameters_combination["l1_ratio"],
                                                C = parameters_combination['C'],
                                                class_weight = dict_class_weight,  max_iter = 1000,n_jobs=-1)

                        model.fit(X_train, y_train)
                        dataframe_results_val = eval_model_compute_metrics(model, X_val, y_val, dataframe_results_val, 
                                                                               n,i,j,parameters_combination, "val","classification")


                elif classifier == "svm":
                    parameters_grid = {'kernel': ['linear','rbf','poly'],
                                        'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 20, 50, 100]}

                    grid_search = ParameterGrid(parameters_grid)

                    for parameters_combination in grid_search:
                        model = SVC(kernel=parameters_combination["kernel"], probability =True,
                                    C=parameters_combination['C'],
                                    class_weight = dict_class_weight, random_state = rn,max_iter = 1000)

                        model.fit(X_train, y_train)
                        dataframe_results_val = eval_model_compute_metrics(model, X_val, y_val, dataframe_results_val, 
                                                                            n,i,j,parameters_combination, "val","classification")

                elif classifier == "random_forest":

                    parameters_grid = {'n_estimators': [100],
                                        'max_depth': [3, 5, 7, 10],
                                        'min_samples_leaf': [1, 3, 5, 10],
                                        'bootstrap':[True, False],
                                        'max_features':[0.25,0.5,0.75],
                                        'criterion':["gini", "entropy"],
                                            }
                    grid_search = ParameterGrid(parameters_grid)

                    for parameters_combination in grid_search:
                        model = RandomForestClassifier(random_state=rn, n_jobs= -1, max_features = parameters_combination["max_features"],
                                                        n_estimators = parameters_combination['n_estimators'],
                                                    max_depth = parameters_combination['max_depth'],
                                                    min_samples_leaf = parameters_combination['min_samples_leaf'],
                                                    bootstrap = parameters_combination['bootstrap'],
                                                    criterion = parameters_combination['criterion'],
                                                        class_weight = dict_class_weight)

                        model.fit(X_train, y_train)
                        dataframe_results_val = eval_model_compute_metrics(model, X_val, y_val, dataframe_results_val, 
                                                                            n,i,j,parameters_combination, "val","classification")

                elif classifier == "xgboost":
                    sample_weight_list = []
                    for sample in y_train:
                        if sample==0:
                            sample_weight_list.append(class_weight[0])
                        elif sample==1:
                            sample_weight_list.append(class_weight[1])

                    parameters_grid = {"learning_rate": [1e-2, 1e-1, 0.3],
                                        "n_estimators": [100],
                                        "gamma": [0.2, 2, 5] ,
                                        "colsample_bytree": [0.25,0.5,0.75],
                                        "max_depth":[3, 5, 7],
                                        "min_child_weight":[1, 3, 5],
                                        "reg_alpha": [10, 1, 0.1, 0.01],
                                        "subsample":[1]}
                    grid_search = ParameterGrid(parameters_grid)

                    for parameters_combination in tqdm(grid_search):

                        model = xgb.XGBClassifier(objective="binary:logistic", random_state=rn, booster="gbtree", n_jobs = -1,
                                                learning_rate = parameters_combination['learning_rate'],
                                                gamma =parameters_combination['gamma'],
                                                reg_alpha =parameters_combination['reg_alpha'],
                                                max_depth = parameters_combination['max_depth'],
                                                min_child_weight = parameters_combination['min_child_weight'],
                                                colsample_bytree = parameters_combination['colsample_bytree'],
                                                n_estimators = parameters_combination['n_estimators'], early_stopping_rounds = 30, eval_metric=['logloss'],
                                                subsample= parameters_combination['subsample'])

                        model.fit(X_train, y_train,eval_set=[(X_val, y_val)], sample_weight=sample_weight_list ,verbose=0 )
                        dataframe_results_val = eval_model_compute_metrics(model, X_val, y_val, dataframe_results_val, 
                                                                            n,i,j,parameters_combination, "val","classification")

                      
            # compute mean validation metrics on the specific inner fold and select best hyperparams
            best_hyperparms_idxmax = dataframe_results_val[dataframe_results_val['inner_fold']==j].groupby('hyperparameters_combination')['validation_mcc'].mean().reset_index()['validation_mcc'].idxmax()
            best_hyperparms = eval(dataframe_results_val.loc[best_hyperparms_idxmax,]['hyperparameters_combination'])
            print(best_hyperparms)
            
            
            
            if source == "clinical_variables":

                # read train_val and test set from the outer fold
                training_val_set = create_label_dataset(pd.read_excel(path_splits + "dataset_train_val_" + str(rn) + "_" + str(i) + ".xlsx"),label)
                test_set = create_label_dataset(pd.read_excel(path_splits + "dataset_test_" + str(rn) + "_" + str(i) + ".xlsx"),label)

                # scale and dummy encodings, define training dataset and outcome
                training_val_set_scaled, test_set_scaled, X_train_val, y_train_val, X_test, y_test = scale_numerical_encode_categorical(training_val_set, test_set, 
                                                        binary_features, categorical_features,continue_features+ordinal_features,
                                                                           "cad_group")
                
            elif source == "gene_expression":
                
                # read train_val and test set from the outer fold
                training_val_set = create_label_dataset(pd.read_excel(path_splits + "dataset_train_val_" + str(rn) + "_" + str(i) + ".xlsx"),label)
                test_set = create_label_dataset(pd.read_excel(path_splits + "dataset_test_" + str(rn) + "_" + str(i) + ".xlsx"),label)

                training_val_set_gen = transpose_and_index_gene_df(pd.read_csv(path_splits + "dataset_gen_train_val_" + str(rn) + "_" + str(i) + '.txt' , sep="\t"))
                test_set_gen = transpose_and_index_gene_df(pd.read_csv(path_splits + "dataset_gen_test_" + str(rn) + "_" + str(i) + '.txt' , sep="\t"))

                training_val_set = training_val_set_gen.merge(training_val_set[['Sample_ID','cad_group']].sort_values(by="Sample_ID"), on ="Sample_ID")
                test_set = test_set_gen.merge(test_set[['Sample_ID','cad_group']].sort_values(by="Sample_ID"), on ="Sample_ID")
    
                gene_expr_features = list(set(training_val_set.columns).difference(set(['Sample_ID','cad_group'])))
    
                # scale and dummy encodings, define training dataset and outcome
                training_val_set_scaled, test_set_scaled, X_train_val, y_train_val, X_test, y_test = scale_numerical_encode_categorical(training_val_set, test_set,  [], [], 
                                                                   gene_expr_features, "cad_group")
            
            
            elif source == "clinical_gene_expression":
            
                # clinical var
                training_val_set = create_label_dataset(pd.read_excel(path_splits + "dataset_train_val_" + str(rn) + "_" + str(i) + ".xlsx"),label)
                test_set = create_label_dataset(pd.read_excel(path_splits + "dataset_test_" + str(rn) + "_" + str(i) + ".xlsx"),label)

                training_val_set_gen = transpose_and_index_gene_df(pd.read_csv(path_splits + "dataset_gen_train_val_" + str(rn) + "_" + str(i) + '.txt' , sep="\t"))
                test_set_gen = transpose_and_index_gene_df(pd.read_csv(path_splits + "dataset_gen_test_" + str(rn) + "_" + str(i) +'.txt' , sep="\t"))

                training_val_set = training_val_set_gen.merge(training_val_set.sort_values(by="Sample_ID"), on ="Sample_ID")
                test_set = test_set_gen.merge(test_set.sort_values(by="Sample_ID"), on ="Sample_ID")

                with open(path_splits + 'gene_selected_DEG_' + str(rn) + "_" + str(i) + '.json', 'r') as file:
                    gene_expr_features = json.load(file)

                training_val_set_scaled, test_set_scaled, X_train_val, y_train_val, X_test, y_test = scale_numerical_encode_categorical(training_val_set, test_set, 
                                                                        binary_features, categorical_features,
                                                                        continue_features + ordinal_features+gene_expr_features,
                                                                        "cad_group")

                if KG_emb:

                    X_train_val = torch.Tensor.numpy(make_patient_representation(training_val_set_scaled, 
                                                                          annotation_embedding,  
                                                                          gene_expr_features,
                                                                          dict_gene_alias_mapping_primeKG,
                                                                          dict_Sample_ID_clinical_annotation_tensor,
                                                                          clinical_features_not_mapped))

                    X_test = torch.Tensor.numpy(make_patient_representation(test_set_scaled, 
                                                                          annotation_embedding,  
                                                                          gene_expr_features,
                                                                          dict_gene_alias_mapping_primeKG,
                                                                          dict_Sample_ID_clinical_annotation_tensor,
                                                                          clinical_features_not_mapped))

            # compute class weight
            class_weight = compute_class_weight('balanced',classes=np.unique(y_train_val), y = y_train_val)
            dict_class_weight = {}
            for c in np.unique(y_train_val):
                dict_class_weight[c] = class_weight[c]
                
            # create the model with the best hyperparams
            if classifier == "logistic_regression":
                model = LogisticRegression(penalty="l1", solver = "liblinear", random_state = rn,
                                                        C = best_hyperparms['C'],
                                                        class_weight = dict_class_weight,
                                                        max_iter = 1000)

                model.fit(X_train_val, y_train_val)
                with open(path_output + classifier + 'best_model_' + str(rn) + '_' + str(i) + '.pkl', 'wb') as file:
                    pickle.dump(model, file)

                dataframe_results_test = eval_model_compute_metrics(model, X_test, y_test, dataframe_results_test, 
                                                                                n,i,j,best_hyperparms, "test","classification")

            elif classifier == "elastic_net":

                model = LogisticRegression(penalty="elasticnet", solver="saga", random_state = rn,
                                                    l1_ratio = best_hyperparms["l1_ratio"],
                                                    C = best_hyperparms['C'],
                                                    class_weight = dict_class_weight,  max_iter = 1000,n_jobs=-1)

                model.fit(X_train_val, y_train_val)
                with open(path_output + classifier + 'best_model_' + str(rn) + '_' + str(i) + '.pkl', 'wb') as file:
                    pickle.dump(model, file)
                dataframe_results_test = eval_model_compute_metrics(model, X_test, y_test, dataframe_results_test, 
                                                                                n,i,j,best_hyperparms, "test","classification")


            elif classifier == "svm":

                model = SVC(kernel=best_hyperparms["kernel"], C = best_hyperparms['C'], probability =True,
                                        class_weight = dict_class_weight, random_state = rn,max_iter = 1000)

                model.fit(X_train_val, y_train_val)
                with open(path_output + classifier + 'best_model_' + str(rn) + '_' + str(i) + '.pkl', 'wb') as file:
                    pickle.dump(model, file)
                dataframe_results_test = eval_model_compute_metrics(model, X_test, y_test, dataframe_results_test, 
                                                                                n,i,j,best_hyperparms, "test","classification")

            elif classifier == "random_forest":


                model = RandomForestClassifier(random_state=rn, n_jobs= -1, max_features = best_hyperparms["max_features"],
                                                    n_estimators = best_hyperparms['n_estimators'],
                                                        max_depth = best_hyperparms['max_depth'],
                                                        min_samples_leaf = best_hyperparms['min_samples_leaf'],
                                                        bootstrap = best_hyperparms['bootstrap'],
                                                        criterion = best_hyperparms['criterion'],
                                                            class_weight = dict_class_weight)

                model.fit(X_train_val, y_train_val)
                with open(path_output + classifier + 'best_model_' + str(rn) + '_' + str(i) + '.pkl', 'wb') as file:
                    pickle.dump(model, file)
                dataframe_results_test = eval_model_compute_metrics(model, X_test, y_test, dataframe_results_test, 
                                                                                n,i,j,best_hyperparms, "test","classification")

            elif classifier == "xgboost":
                sample_weight_list = []
                for sample in y_train_val:
                    if sample==0:
                        sample_weight_list.append(class_weight[0])
                    elif sample==1:
                        sample_weight_list.append(class_weight[1])


                model = xgb.XGBClassifier(objective="binary:logistic", random_state=rn, booster="gbtree", n_jobs = -1,
                                                    learning_rate = best_hyperparms['learning_rate'],
                                                    gamma =best_hyperparms['gamma'],
                                                    reg_alpha =best_hyperparms['reg_alpha'],
                                                    max_depth = best_hyperparms['max_depth'],
                                                    min_child_weight = best_hyperparms['min_child_weight'],
                                                    colsample_bytree = best_hyperparms['colsample_bytree'],
                                                    n_estimators = best_hyperparms['n_estimators'],
                                                    subsample= best_hyperparms['subsample'])

                model.fit(X_train_val, y_train_val, sample_weight=sample_weight_list ,verbose=0 )
                with open(path_output + classifier + 'best_model_' + str(rn) + '_' + str(i) + '.pkl', 'wb') as file:
                    pickle.dump(model, file)
                dataframe_results_test = eval_model_compute_metrics(model, X_test, y_test, dataframe_results_test, 
                                                                                n,i,j,best_hyperparms, "test","classification")

       
        dataframe_results_test.to_excel(path_output + classifier + "_df_test_results.xlsx",index=False)


# Training

### <font color='red'> Clinical features </font> 

In [ ]:
for cl in ["logistic_regression", "elastic_net","svm", "random_forest","xgboost"]:
    ML_classification(127, cl, "./all_data/m25/", 1, "./results_all_data/m25/clinical_variables/", "m2vsall", 10, 'clinical_variables', False,None)
    ML_classification(127, cl, "./all_data/m50/", 1, "./results_all_data/m50/clinical_variables/", "m3vsall", 10, 'clinical_variables', False,None)
    ML_classification(127, cl, "./all_data/m70/", 1, "./results_all_data/m70/clinical_variables/", "4a4b5vsall", 10, 'clinical_variables', False,None)

### <font color='green'> Gene Expression </font> 

In [ ]:
for cl in ["logistic_regression", "elastic_net","svm", "random_forest","xgboost"]:
    ML_classification(127, cl, "./all_data/m25/", 1, "./results_all_data/m25/", "m2vsall", 10, "gene_expression", False,None)
    ML_classification(127, cl, "./all_data/m50/", 1, "./results_all_data/m50/", "m3vsall", 10, "gene_expression", False,None)
    ML_classification(127, cl, "./all_data/m70/", 1, "./results_all_data/m70/", "4a4b5vsall", 10, "gene_expression", False,None)
    

### <font color='blue'> Clinical variables and gene expression </font> 

In [ ]:
for cl in ["logistic_regression", "elastic_net","svm", "random_forest","xgboost"]:
    ML_classification(127, cl, "./all_data/m25/", 1, "./results_all_data/m25/", "m2vsall", 10, "clinical_gene_expression",False,None)
    ML_classification(127, cl, "./all_data/m50/", 1, "./results_all_data/m50/", "m3vsall", 10, "clinical_gene_expression",False,None)
    ML_classification(127, cl, "./all_data/m70/", 1, "./results_all_data/m70/", "4a4b5vsall", 10, "clinical_gene_expression",False,None)


### Clinical variables and gene expression with <font color='blue'> KG emb </font> 

In [ ]:
for cl in ["logistic_regression", "elastic_net","svm", "random_forest","xgboost"]:

    ML_classification(127, cl, "./all_data/m25/", 1, "./results_all_data/m25/KGembPCA_clinical_gene_DGE/", "m2vsall", 10, 
                    "clinical_gene_expression","DGE_already_performed",True,best_RotatE_embeddings_entities)
    ML_classification(127, cl, "./all_data/m50/", 1, "./results_all_data/m50/KGembPCA_clinical_gene_DGE/", "m3vsall", 10, 
                    "clinical_gene_expression","DGE_already_performed",True,best_RotatE_embeddings_entities)
    ML_classification(127, cl, "./all_data/m70/", 1, "./results_all_data/m70/KGembPCA_clinical_gene_DGE/", "4a4b5vsall", 10, 
                    "clinical_gene_expression","DGE_already_performed",True,best_RotatE_embeddings_entities)

# Read results

In [ ]:

sources = ['clinical_variables', 'gene_expression_DGE', 'clinical_gene_DGE','KG_emb_clinical_gene_DGE']

test_metrics = ['test_accuracy', 'test_wROCAUC', 'test_mcc', 'test_wPRAUC', 'test_wF1', 'test_brier',
                'test_sens_rec', 'test_spec','test_prec_ppv', 'test_npv']

df_results_test_plot = pd.DataFrame(columns = ["label","source","model"] + test_metrics )
df_mean_std_test = pd.DataFrame(columns = ["label","source","model"] + test_metrics )

for source_ in sources:

    for label in ["m25","m50","m70"]:

        for model_name in ["logistic_regression","elastic_net","svm","random_forest","xgboost"]:

            path_test_results = "./results_all_data/" + label + "/" + source_ + "/" + model_name + "_df_test_results.xlsx"

            classifier_test_results = pd.read_excel(path_test_results)
            classifier_test_results['model'] = [model_name for x in range(len(classifier_test_results))]
            classifier_test_results['label'] = [label for x in range(len(classifier_test_results))]
            classifier_test_results['source'] = [source_ for x in range(len(classifier_test_results))]

            mean_df_model = np.mean(classifier_test_results[test_metrics],axis=0)
            std_df_model = np.std(classifier_test_results[test_metrics],axis=0)

            new_row = pd.Series({"label":label,
                                 "model":model_name,
                                 "test_accuracy":np.round(mean_df_model['test_accuracy'],3).astype(str) + "±" + np.round(std_df_model['test_accuracy'],3).astype(str) ,
                                 "test_wROCAUC":np.round(mean_df_model['test_wROCAUC'],3).astype(str) + "±" + np.round(std_df_model['test_wROCAUC'],3).astype(str) ,
                                 "test_mcc":np.round(mean_df_model['test_mcc'],3).astype(str) + "±" + np.round(std_df_model['test_mcc'],3).astype(str) ,
                                 "test_wPRAUC":np.round(mean_df_model['test_wPRAUC'],3).astype(str) + "±" + np.round(std_df_model['test_wPRAUC'],3).astype(str) ,
                                 "test_wF1":np.round(mean_df_model['test_wF1'],3).astype(str) + "±" + np.round(std_df_model['test_wF1'],3).astype(str) ,
                                 "test_brier": np.round(mean_df_model['test_brier'],3).astype(str) + "±" + np.round(std_df_model['test_brier'],3).astype(str) ,
                                 "test_sens_rec":np.round(mean_df_model['test_sens_rec'],3).astype(str) + "±" + np.round(std_df_model['test_sens_rec'],3).astype(str) ,
                                 "test_spec":np.round(mean_df_model['test_spec'],3).astype(str) + "±" + np.round(std_df_model['test_spec'],3).astype(str) ,
                                 "test_prec_ppv":np.round(mean_df_model['test_prec_ppv'],3).astype(str) + "±" + np.round(std_df_model['test_prec_ppv'],3).astype(str) ,
                                 "test_npv":np.round(mean_df_model['test_npv'],3).astype(str) + "±" + np.round(std_df_model['test_npv'],3).astype(str) ,
                                 "source":source_})
            df_mean_std_test =   pd.concat([df_mean_std_test, new_row.to_frame().T], ignore_index=True)
            df_results_test_plot = pd.concat([df_results_test_plot,classifier_test_results[["label","source","model"] + test_metrics]])

df_mean_std_test.to_excel("./CAS_prediction_results.xlsx",index=False)